In [71]:
import sys
import loadOptions as lp


def loadFiles(version, arena):

    file_path     = "../data/"
    list_of_files = []
    arena_list    = ["arena-1",
                     "arena-2",
                     "arena-3",
                     "arena-4",
                     "arena-5",
                     "arena-6",
                     "arena-7",
                     "arena-8",
                     "arena-9",]

    file_path = getVersionPath(version, file_path)

    if file_path == None:
        
        sys.exit(1)

    else:
        
        if arena in arena_list:
            
            file_path        += "/" + arena + "/"
            encounter_pokemon = file_path + "EncounterablePokemon.csv"
            pokemon_of_leader = file_path + "LeaderPokemon.csv"
            arena_leader = file_path + "ArenaLeader.csv"
            move_of_in        = file_path + "MovesOfInterest.csv"

            list_of_files.append(encounter_pokemon)
            list_of_files.append(pokemon_of_leader)
            list_of_files.append(arena_leader)
            list_of_files.append(move_of_in)
            
        else:

            print("[-] Arena not found...")



    return list_of_files

def getVersionPath(version, file_path):

    if version == "red":

        file_path += "red"

        return file_path

    elif version == "blue":

        file_path += "blue"
    
        return file_path

    else:

        print("[-] Version not found...")



In [275]:
csvPaths = loadFiles("blue", "arena-1")

In [287]:
# TODO: Arena-9 fragen was blue spalten bedeuten
def loadStones(arena):
    #[
    #                  "moon-stone",
    #                  "fire-stone",
    #                  "thunder-stone",
    #                  "water-stone",
    #                  "leaf-stone",
    #                  ]

    list_of_stones = []
    
    arena1_stones = []
    arena2_stones =  arena1_stones + ["moon-stone"]
    arena3_stones = arena2_stones + []
    #... arena 4 - 9

    # Starter befor arena 1
    # Walk befor arean 1
    #
    # Moon after arena 2
    # Other stones arena 3
    #
    # Fishing 1 after arena 2
    # Fishing 2 after arena 3
    # Fishing 3 after arena 3

    if arena == "arena-1":
        pass
    elif arena == "arena-2":
        for stone in arena2_stones:  
            list_of_stones.append(stone)
    elif arena == "arena-3":
        for stone in arena3_stones:
            list_of_stones.append(stone)
    return list_of_stones




In [316]:
# SETUP DATA
ARENA = 'arena-1'
GAME = 'blue'
csvPaths = loadFiles(GAME, ARENA)


encounterable_pokemonTable = pd.read_csv(csvPaths[0])
encounterable_pokemonParsed = []

for row in encounterable_pokemonTable.values.tolist():
    moves = row[2]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[2] = moves
    encounterable_pokemonParsed.append(row)
    
leader_pokemon =  pd.read_csv(csvPaths[1])

#test = leader_pokemon.loc[leader_pokemon['name']=='geodude']['moves'].values.tolist()[0].replace("'", "\"") 
#json.loads(test)


arena_leader = pd.read_csv(csvPaths[2])
arena_leader_parsed = []
for row in arena_leader.values.tolist():
    moves = row[3]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[3] = moves
    arena_leader_parsed.append(row)

moves_of_interest = pd.read_csv(csvPaths[3])

## extracted Variables from DATAFRAME
averageLevel = arena_leader['averageLevel'][0]

## Variables for each Arena
arena1_Options = loadStones(ARENA)


In [277]:
# spuckt liste von Pokemon aus welche zur verf. stehen

# @para1: encounterable_pokemon - liste von antreffbaren pokemon :: [index,name,encounterOrEvolutionTrigger,...]
# @para2: averageLevel - averageLevel des ArenaLeiters
# @para3: stones_available - Die Entw. Steine welche zur verfuegung stehen
# @return: poss_team_member - liste von pokemon welche mit dem averageLevel und den vorhanden Steinen erreichbar sind :: [index,name,encounterOrEvolutionTrigger,...]
def possPokemonArenaNLeaderM(encounterable_pokemon, averageLevel, stones_available):
    
    encounterable_pokemon_available = []
    for pokemon in encounterable_pokemon:
        encounterOrEvolutionTrigger = pokemon[3]
        if encounterOrEvolutionTrigger.isnumeric() and int(encounterOrEvolutionTrigger) <= averageLevel:
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger == 'walk' or encounterOrEvolutionTrigger == 'starter' or encounterOrEvolutionTrigger == 'special' or encounterOrEvolutionTrigger == 'trade':
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger in stones_available:
            encounterable_pokemon_available.append(pokemon)
    return encounterable_pokemon_available

In [279]:
possPokemonArenaNLeaderM(encounterable_pokemonParsed, averageLevel, arena1_Options)

[[0,
  'pidgey',
  [{'name': 'gust', 'levelLearnedAt': 1},
   {'name': 'sand-attack', 'levelLearnedAt': 5},
   {'name': 'quick-attack', 'levelLearnedAt': 12},
   {'name': 'whirlwind', 'levelLearnedAt': 19},
   {'name': 'wing-attack', 'levelLearnedAt': 28},
   {'name': 'agility', 'levelLearnedAt': 36},
   {'name': 'mirror-move', 'levelLearnedAt': 44}],
  'walk',
  nan,
  '[]',
  '[]'],
 [1,
  'rattata',
  [{'name': 'tackle', 'levelLearnedAt': 1},
   {'name': 'tail-whip', 'levelLearnedAt': 1},
   {'name': 'quick-attack', 'levelLearnedAt': 7},
   {'name': 'hyper-fang', 'levelLearnedAt': 14},
   {'name': 'focus-energy', 'levelLearnedAt': 23},
   {'name': 'super-fang', 'levelLearnedAt': 34}],
  'walk',
  nan,
  '[]',
  '[]'],
 [2,
  'caterpie',
  [{'name': 'tackle', 'levelLearnedAt': 1},
   {'name': 'string-shot', 'levelLearnedAt': 1}],
  'walk',
  nan,
  '[]',
  '[]'],
 [3,
  'metapod',
  [{'name': 'harden', 'levelLearnedAt': 1}],
  'walk',
  nan,
  '[]',
  '[]'],
 [4,
  'weedle',
  [{'nam

In [ ]:
# Diese func. gibt eine list von Moves, von encounter_pokemon zurueck.
# @para1: poss_team_member - DataFrame von moeglichen Team-Membern
# @para2: name - name des Pokemon. Dieser wird benoetigt um seine moves zu suchen.
# @para3: lock - ist ein schloss fuer das anfuegen der Moves an einer liste 
# @return: move_list - gibt eine liste der moeglichen moves zurueck.

def getPokemonMoves(poke_team, name, lock):
    # list der moves eines Pokemon
    move_list = []

    poke = poke_team.loc[poke_team["name"] == name, ["moves"]]
    poke = poke.values[0][0]
    poke = poke.replace("'", "\"") 
    poke = json.loads(poke)
    
    # Die Tabellen EncounterablePokemon und LeaderPokemon speichern die Moves
    # unterschiedlich. Bei der EncounterablePokemon ist in der Spalte 'Moves' ein
    # weiters DataFrame enthalten, wo die moves der Pokemon unter dem Value 'name'
    # gespeichert sind. Deshalb muessen das auslesen dafuer etwas anders gemacht werden
    # Wenn lock == 0 ist werden die Moves unsers Teams der List hinzugefuegt 
    # Wenn lock == 1 ist werden die Moves des Arena-Teams der list hinzugefuegt
    if lock == 0:
        for move in poke:
            move_list.append(move["name"])

    elif lock == 1:

        for move in poke:
            move_list.append(move)
            
    return move_list